# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ts" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "jw300-baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [0]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-ts-jw300-baseline


In [0]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 2.8MB/s 


In [0]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ts.xml.gz not found. The following files are available for downloading:

   7 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-ts.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  96 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/ts.zip

 366 MB Total size
./JW300_latest_xml_en-ts.xml.gz ... 100% of 7 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_ts.zip ... 100% of 96 MB


In [0]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-01-12 07:56:31--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.04s   

2020-01-12 07:56:31 (6.25 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-01-12 07:56:33--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-ts.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [0]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [0]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 7170/853024 lines since contained in test set.


,source_sentence,target_sentence
0,Dowsing ​ — Scientific or Occult ?,Ku Femba — I Sayense Kumbe I Vungoma ?
1,“ AMAZING ! ”,“ WA hlamarisa ! ”
2,exclaimed a dairy farmer in the Midwestern Uni...,ku huwelela n’wamapursi wa vufuwela - masi le ...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [0]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 2.0MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144668 sha256=af7227e367da5ab44d12864a032388d1f91ad66813b6324f428ad4d4051d276e
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.32 0.00 percent complete
00:00:23.27 0.13 percent complete
00:00:47.22 0.26 percent complete
00:01:10.63 0.39 percent complete
00:01:34.28 0.52 percent complete
00:01:57.40 0.65 percent complete
00:02:20.49 0.78 percent complete
00:02:43.69 0.91 percent complete
00:03:07.31 1.04 percent complete
00:03:30.38 1.17 percent complete
00:03:54.32 1.30 percent complete
00:04:17.98 1.43 percent complete
00:04:41.40 1.56 percent complete
00:05:05.13 1.69 percent complete


00:05:28.89 1.82 percent complete
00:05:52.75 1.95 percent complete


00:06:15.81 2.07 percent complete


00:06:39.30 2.20 percent complete
00:07:01.96 2.33 percent complete
00:07:25.97 2.46 percent complete
00:07:49.50 2.59 percent complete
00:08:12.57 2.72 percent complete
00:08:35.87 2.85 percent complete
00:08:58.55 2.98 percent complete
00:09:21.65 3.11 percent complete
00:09:44.55 3.24 percent complete
00:10:08.15 3.37 percent complete


00:10:32.04 3.50 percent complete
00:10:54.97 3.63 percent complete
00:11:18.58 3.76 percent complete
00:11:42.29 3.89 percent complete
00:12:05.27 4.02 percent complete
00:12:28.40 4.15 percent complete
00:12:51.70 4.28 percent complete
00:13:14.65 4.41 percent complete
00:13:38.47 4.54 percent complete
00:14:02.18 4.67 percent complete
00:14:26.14 4.80 percent complete
00:14:49.12 4.93 percent complete
00:15:12.18 5.06 percent complete
00:15:35.69 5.19 percent complete
00:15:59.03 5.32 percent complete
00:16:22.66 5.45 percent complete
00:16:46.44 5.58 percent complete
00:17:10.38 5.71 percent complete
00:17:34.42 5.84 percent complete
00:17:57.87 5.96 percent complete
00:18:20.91 6.09 percent complete
00:18:44.01 6.22 percent complete
00:19:07.93 6.35 percent complete
00:19:31.96 6.48 percent complete
00:19:55.89 6.61 percent complete
00:20:18.82 6.74 percent complete
00:20:42.58 6.87 percent complete
00:21:05.97 7.00 percent complete
00:21:29.15 7.13 percent complete
00:21:52.31 7.

...

04:19:36.56 86.10 percent complete
04:19:59.76 86.23 percent complete
04:20:23.04 86.36 percent complete
04:20:47.07 86.49 percent complete
04:21:10.30 86.62 percent complete
04:21:33.90 86.75 percent complete
04:21:57.86 86.88 percent complete
04:22:21.61 87.01 percent complete
04:22:44.91 87.14 percent complete
04:23:07.81 87.27 percent complete
04:23:31.42 87.40 percent complete
04:23:55.14 87.53 percent complete
04:24:18.65 87.66 percent complete
04:24:41.65 87.79 percent complete
04:25:05.42 87.92 percent complete
04:25:28.31 88.05 percent complete
04:25:52.43 88.18 percent complete
04:26:16.07 88.31 percent complete
04:26:39.40 88.44 percent complete
04:27:03.59 88.57 percent complete
04:27:27.28 88.70 percent complete
04:27:51.04 88.83 percent complete
04:28:14.16 88.96 percent complete
04:28:37.25 89.09 percent complete
04:29:00.93 89.21 percent complete


04:29:24.66 89.34 percent complete
04:29:48.37 89.47 percent complete
04:30:11.23 89.60 percent complete
04:30:34.72 89.73 percent complete
04:30:58.39 89.86 percent complete
04:31:20.75 89.99 percent complete
04:31:44.40 90.12 percent complete
04:32:07.43 90.25 percent complete
04:32:30.81 90.38 percent complete
04:32:53.90 90.51 percent complete


04:33:17.16 90.64 percent complete
04:33:40.93 90.77 percent complete
04:34:04.58 90.90 percent complete
04:34:28.06 91.03 percent complete
04:34:51.43 91.16 percent complete
04:35:15.71 91.29 percent complete
04:35:40.02 91.42 percent complete
04:36:03.36 91.55 percent complete
04:36:27.82 91.68 percent complete
04:36:52.06 91.81 percent complete
04:37:14.68 91.94 percent complete
04:37:38.26 92.07 percent complete
04:38:01.50 92.20 percent complete
04:38:24.92 92.33 percent complete
04:38:48.35 92.46 percent complete
04:39:10.99 92.59 percent complete
04:39:35.08 92.72 percent complete
04:39:58.58 92.85 percent complete
04:40:22.30 92.98 percent complete
04:40:45.63 93.11 percent complete
04:41:09.35 93.23 percent complete
04:41:33.12 93.36 percent complete
04:41:56.72 93.49 percent complete
04:42:19.88 93.62 percent complete
04:42:43.58 93.75 percent complete
04:43:08.11 93.88 percent complete
04:43:31.80 94.01 percent complete
04:43:55.84 94.14 percent complete
04:44:18.50 94.27 pe

04:48:35.07 95.70 percent complete
04:48:58.40 95.83 percent complete
04:49:22.59 95.96 percent complete
04:49:45.14 96.09 percent complete
04:50:08.70 96.22 percent complete
04:50:31.72 96.35 percent complete
04:50:55.85 96.48 percent complete
04:51:18.94 96.61 percent complete
04:51:42.51 96.74 percent complete
04:52:05.43 96.87 percent complete
04:52:29.10 97.00 percent complete
04:52:52.33 97.12 percent complete
04:53:15.84 97.25 percent complete
04:53:39.87 97.38 percent complete
04:54:03.54 97.51 percent complete
04:54:26.86 97.64 percent complete
04:54:50.21 97.77 percent complete
04:55:13.23 97.90 percent complete
04:55:36.53 98.03 percent complete
04:56:00.25 98.16 percent complete
04:56:23.56 98.29 percent complete
04:56:47.46 98.42 percent complete


04:57:10.69 98.55 percent complete
04:57:34.57 98.68 percent complete
04:57:58.16 98.81 percent complete
04:58:22.14 98.94 percent complete
04:58:46.25 99.07 percent complete
04:59:09.46 99.20 percent complete
04:59:33.09 99.33 percent complete
04:59:56.97 99.46 percent complete
05:00:20.35 99.59 percent complete
05:00:43.63 99.72 percent complete
05:01:07.55 99.85 percent complete
05:01:30.90 99.98 percent complete


In [0]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
However , the real cause of their distress went far deeper .
If they dig down into Sheol , from there my own hand will take them ; and if they go up to the heavens , from there I shall bring them down . ” ​ — Amos 9 : 1 , 2 .
Through his sacred Word , the Holy Bible , God has made clear what his will is .
We remembered Jesus ’ admonition to his disciples that they should “ keep bearing much fruit . ”
The lower back dial predicted solar and lunar eclipses
Most of the time , I traveled from one congregation to the next by bus or by train .
The challenge is to be happy as well as organized in all that we do .
Some spouses are holding down two jobs just to get by , and in other cases both spouses are working , leaving the children with grandparents or at a child - care center .
We can see that from the resurrections Jesus performed when outside of Nain and when in the home of Jairus .
Another way we can honor Jehovah God is by making monetary contributions to the worldwide



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [0]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 2199 (delta 4), reused 5 (delta 3), pack-reused 2184
Receiving objects: 100% (2199/2199), 2.60 MiB | 16.87 MiB/s, done.
Resolving deltas: 100% (1525/1525), done.
Processing /content/joeynmt
     |████████████████████████████████| 276kB 9.6MB/s 
     |████████████████████████████████| 307kB 42.3MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 215kB 43.6MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 737kB 44.3MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=6ef7b5ec1557e6e06124618e158d2ad7a5e9889d63b8b580388f0e1d88790ba6
  Stored in directory: /tmp/pip-ephem-wheel-cache-bnh801j_/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyaml

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Xitsonga Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.ts     test.ts	   train.en
dev.bpe.en	dev.ts	     test.en	     train.bpe.en  train.ts
dev.bpe.ts	test.bpe.en  test.en-any.en  train.bpe.ts
bpe.codes.4000	dev.en	     test.bpe.ts     test.ts	   train.en
dev.bpe.en	dev.ts	     test.en	     train.bpe.en  train.ts
dev.bpe.ts	test.bpe.en  test.en-any.en  train.bpe.ts
BPE Xitsonga Sentences
( Hlaya Luka 21 : 34 , 35 . )
I yini leswi nga endleka eka Petro , Yakobo na Yohane leswi nga endle@@ kaka ni le ka hina ?
Hi ku ya hi Luka 21 : 36 , Yesu u hi dyondzisa njhani ku ‘ tshama hi h@@ it@@ ekile ’ ?
Hi nga tiyiseka njhani leswaku hi swi lungh@@ ek@@ erile ku langutana ni leswi nga ta endleka ku nga ri khale ?
[ 1 ] ( ndz@@ imana 14 ) Vona ndz@@ ima 21 ya buku leyi nge , Mfumo Wa Xikwembu Wa F@@ u@@ ma !
Combined BPE Vocab
ά@@
;@@
õ@@
τ@@
̭@@
̌
avulo@@
Ο@@
×
̱


In [0]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.ts     test.ts	   train.en
dev.bpe.en	dev.ts	     test.en	     train.bpe.en  train.ts
dev.bpe.ts	test.bpe.en  test.en-any.en  train.bpe.ts


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [0]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-01-12 13:08:59,113 Hello! This is Joey-NMT.
2020-01-12 13:09:00,527 Total params: 12201472
2020-01-12 13:09:00,528 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

cp: cannot create symbolic link '/content/drive/My Drive/masakhane/en-ts-jw300-baseline/models/ents_transformer/best.ckpt': Operation not supported


In [0]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 115859.95312	PPL: 50.56873	bleu: 0.28609	LR: 0.00030000	*
Steps: 2000	Loss: 93454.92188	PPL: 23.68017	bleu: 1.25480	LR: 0.00030000	*
Steps: 3000	Loss: 82517.15625	PPL: 16.35045	bleu: 3.86151	LR: 0.00030000	*
Steps: 4000	Loss: 74959.91406	PPL: 12.65873	bleu: 7.51147	LR: 0.00030000	*
Steps: 5000	Loss: 69489.76562	PPL: 10.51826	bleu: 10.79874	LR: 0.00030000	*
Steps: 6000	Loss: 65908.36719	PPL: 9.31696	bleu: 12.64414	LR: 0.00030000	*
Steps: 7000	Loss: 62382.48438	PPL: 8.26840	bleu: 14.44985	LR: 0.00030000	*
Steps: 8000	Loss: 59565.63672	PPL: 7.51615	bleu: 16.71410	LR: 0.00030000	*
Steps: 9000	Loss: 56864.46094	PPL: 6.85916	bleu: 19.01188	LR: 0.00030000	*
Steps: 10000	Loss: 54950.11719	PPL: 6.42862	bleu: 19.83296	LR: 0.00030000	*
Steps: 11000	Loss: 54042.69531	PPL: 6.23409	bleu: 20.71138	LR: 0.00030000	*
Steps: 12000	Loss: 51465.35938	PPL: 5.71307	bleu: 22.63582	LR: 0.00030000	*
Steps: 13000	Loss: 50235.17969	PPL: 5.47997	bleu: 23.31744	LR: 0.00030000	*
Steps: 14000	Loss: 

In [0]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2020-01-12 16:24:28,116 Hello! This is Joey-NMT.
2020-01-12 16:25:16,568  dev bleu:  35.07 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-01-12 16:26:05,704 test bleu:  44.15 [Beam search decoding with beam size = 5 and alpha = 1.0]
